In [1]:
import numpy as np
import torch
import torch.nn as nn
import os

import value_net

data_dir = f'{os.getcwd()}/data'
db_white_path = f'{data_dir}/train/white.txt'
db_black_path = f'{data_dir}/train/black.txt'
db_draw_path = f'{data_dir}/train/draw.txt'

In [2]:
model = value_net.Net()

loss_history = []
acc_history = []

In [3]:
epochs = 1000
batch_size = 256
lr = .0001
betas=(0.9, 0.99)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)

sampler = value_net.Sampler(db_white_path, db_black_path, db_draw_path)

In [5]:
import tqdm

def epoch(model):
    opt.zero_grad()
    
    x, target = sampler.gen_uniform_sample(batch_size)

    y = model(x)
    loss = loss_fn(y, target)
    
    with torch.no_grad():
        y_labels = torch.argmax(y, -1)
        t_labels = torch.argmax(target, -1)

        correct_labels = torch.eq(y_labels, t_labels)
        accuracy = correct_labels.sum() / correct_labels.numel()

    loss.backward()
    opt.step()

    return loss.item(), accuracy.item()

t = tqdm.trange(epochs)
for i in t:
    loss, acc = epoch(model)
    loss_history.append(loss)
    acc_history.append(acc)

    t.set_description(f'Loss: {loss}, Acc: {acc}')
    t.refresh()

Loss: 1.0963331460952759, Acc: 0.3359375:   1%|          | 12/1000 [00:55<1:15:55,  4.61s/it]


KeyboardInterrupt: 